# Experiment and path specification

In [ ]:
# Specify the experiment for analysis
experiment='cxix40218'
runNumber = 58

# Set the installation and output path
import os
os.environ['INSTALLPATH']= '/cds/home/i/igabalsk/TRXS-Run18'
os.environ['OUTPUTPATH']= '/cds/data/psdm/%s/%s/scratch' % (experiment[0:3],experiment)

# Import Libraries

In [ ]:
# Magic iPython command to enable plotting
%matplotlib inline

# Load in the pythonBatchMagic library
import sys
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/pythonBatchMagic')
from pythonBatchMagic import *

# Determine current user
currentUser, error = unixCMD("echo $USER")
currentUser = currentUser.strip()
print(currentUser)
os.environ['RESULTSPATH']= ('/cds/data/psdm/%s/%s/results/%s' % (experiment[0:3],experiment,currentUser)).strip()
if not os.path.exists(os.environ['RESULTSPATH']):
    os.mkdir(os.environ['RESULTSPATH'])

# Leveraging the batch queue to quickly grab point data

## Loading libraries

In [ ]:
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/LCLS')
from LCLSdefault import *

sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/mattsLibraries')
from picklez import *

# Load in the get data library
from dataAnalysis import *

# Load in the batch library for lcls
from lclsBatch import *

## Grabbing point data across nodes

In [ ]:
detDict = {
#     'uvint':{'name': 'Acqiris','get-function':'getAcqirisSum2'},
#            'uvint1':{'name': 'Acqiris','get-function':'getAcqirisSum1'},
           'xrayEnergy':{'name':'FEEGasDetEnergy','get-function': 'getXrayEnergy' },
           'ebeamcharge':{'name':'EBeam', 'get-function': 'getEbeamCharge'},
          'stageencoder':{'name':'CXI:LAS:MMN:04.RBV','get-function':'getStageEncoder'},
          'ttfltpos':{'name':'CXI:TIMETOOL:FLTPOS', 'get-function': 'getTTFltPos'},
          'vitpos':{'name':'LAS:FS5:VIT:FS_TGT_TIME_DIAL', 'get-function': 'getTTFltPos'},
           'xint':{'name':'CxiDg3_Imb01','get-function': 'getIPM' },
           'xpos':{'name':'CxiDg3_Imb01','get-function': 'getXPos' },
           'ypos':{'name':'CxiDg3_Imb01','get-function': 'getYPos' },
          'seconds':{'name':'seconds', 'get-function': 'getSeconds'},
          'nanoseconds':{'name':'nanonseconds', 'get-function': 'getNanoseconds'},
          'fiducials':{'name':'fiducials', 'get-function': 'getFiducials'}
}
print detDict.keys()
NSHOTS = 100000
ngrab = 500
rank = int( float(NSHOTS)/float(ngrab) )
print('Grabbing %d shots, %d events at a time, on %d nodes' %(NSHOTS,ngrab,rank))
batchThreads = batchPointDataGrabber( detDict, eventMax=int(rank*ngrab), experiment=experiment, runNumber=runNumber, rank=int(rank) )
# batchThreads.Queue = 'psanaq'
batchThreads.Queue = 'psfehhiprioq'
batchThreads.RunType = 'mpirun python2'
batchThreads.start()

## Check job status

In [ ]:
stdoutdata, stderrdata = bjobs()
print stdoutdata
print stderrdata

## Kill threads if they die

In [ ]:
batchThreads.requestStop()
time.sleep(5)
stdoutdata, stderrdata = bkill(killAll=True)
print stdoutdata
print stderrdata

In [ ]:
batchThreads.status

In [ ]:
batchThreads.gather()

## Drop empty array values

In [ ]:
pointData = batchThreads.detArrays

def dropEmpty(pointData):

    filledBin = (pointData['seconds'] > 0)    

    for key in pointData.keys():
        pointData[key]=pointData[key][filledBin] 
        
    return pointData

def dropNaN(pointData):
    goodBin = np.ones(pointData['seconds'].shape, dtype=bool)
    for key in pointData.keys():
        goodBin = goodBin & ~np.isnan(pointData[key])
    for key in pointData.keys():
        pointData[key]=pointData[key][goodBin]
    return pointData
print len(pointData['seconds'])
pointData_masked = dropNaN(dropEmpty(pointData))
print 'Number of good shots: ', len(pointData_masked['seconds'])

## Save point data

In [ ]:
from scipy.io import savemat
save_obj( pointData_masked, os.environ['RESULTSPATH']+'/pointData-run-%d' % runNumber )
savemat(os.environ['RESULTSPATH']+'/pointData-run-%d' % runNumber, pointData_masked)

## Plot histogram of result

In [ ]:
detArrays = pointData_masked

# print detArrays['ttfltpos'].min()
fig2, ax2 = plt.subplots(1, 1)
goodTT = detArrays['ttfltpos'][detArrays['ttfltpos']<3]
ax2.hist(detArrays['ttfltpos'], bins=500,normed=False, alpha=0.5);